In [ ]:
# This block throws many errors like [CONSOLE]. ℹ Console error: Cannot redefine property: webdriver
# Does not affect final output. That's Playwright throwing some errors while scraping webpage
async def web_scraping_webpage(url, magic = True, simulate_user = True):
    browser_config = BrowserConfig(
        browser_type="chromium",  # Options: "chromium", "firefox", "webkit"
        headless=True,            # Set to False if you want to see the browser window
        viewport_width=1280,
        viewport_height=800,
        verbose=True,
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    )
    async with AsyncWebCrawler(config=browser_config) as crawler:
        crawler_config = CrawlerRunConfig(
            screenshot=False,
            verbose=True,
            cache_mode=CacheMode.DISABLED,
            wait_until='domcontentloaded',
            delay_before_return_html = 5,
            log_console=True,
            exclude_social_media_links=True,
            # exclude_external_links = True,
            exclude_external_images  =True,
            magic=magic,
            simulate_user=simulate_user,
            scan_full_page=True,  # Tells the crawler to try scrolling the entire page
            scroll_delay=1,     # Delay (seconds) between scroll steps
        )
        result = None
        try:
            result = await crawler.arun(
                url=url,
                config=crawler_config
            )
        except Exception as e:
        # if not result.success:
            print(f"Crawl Exception: {e}")
        if result is None or not getattr(result, 'markdown', None):
            print(f"Crawl failed or no markdown for {url}")
            return ['URL not found']

        # print(result.markdown)
        return result.markdown
        # print(lines)
        # print(result.html)



nest_asyncio.apply()

async def get_ingredient_list(user_query):
  url = get_webpage_url(user_query)
  ingredient_list_str = ""
  # url = get_webpage_url("Super Cream With Sunscreen SPF 30")
  print(url)
  # url = "https://incidecoder.com/products/olay-super-cream-with-sunscreen-spf-30"
  # url = "https://www.skinsafeproducts.com/l-oreal-paris-colour-riche-lipstick-mica-0-13-oz"
  if "incidecoder.com" in url:
      processed_rows = asyncio.run(web_scraping_webpage(url, magic = True, simulate_user = True))
      # print(processed_rows)
      parts = str(processed_rows).split("## Ingredients overview")
      # print(parts[0])
      if "[less]" in parts[1]:
          ingredient_list_with_description = parts[1].split("[less]")[0]
      else: ingredient_list_with_description = ""
      matches = re.findall(r'\[(.*?)\]', ingredient_list_with_description)
      ingredient_list_wo_description = []
      i = 0
      for each_ingredient in matches:
          ingredient_list_wo_description.append((str(i+1) + ":" + each_ingredient))
          i += 1
      ingredient_list_str = ""
      print(f"There are a total of {i} ingredients, as listed on INCIdecoder: \n")
      ingredient_list_str += f"There are a total of {i} ingredients, as listed on INCIdecoder: \n"

      for each_ingredient in ingredient_list_wo_description:
          print(f"{str(each_ingredient)} ")
          ingredient_list_str += f"{str(each_ingredient)}\n"
  else:
      print(f"Visit {url} to read ingredients list.")
      ingredient_list_str += f"Visit {url} to read ingredients list."
  return ingredient_list_str
# await get_ingredient_list("The Body Shop Camomile Sumptuous Cleansing Butter")